<a href="https://colab.research.google.com/github/JyotikaJayani-08/ThereForYou/blob/main/SuicideDectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm



In [ ]:
data=pd.read_csv('Suicide_Ideation_Dataset(Twitter-based).csv')
data.head()

,Tweet,Suicide
0,making some lunch,Not Suicide post
1,@Alexia You want his money.,Not Suicide post
2,@dizzyhrvy that crap took me forever to put to...,Potential Suicide post
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,Not Suicide post
4,Trying out &quot;Delicious Library 2&quot; wit...,Not Suicide post


In [ ]:
suicide_counts = data['Suicide'].value_counts()

In [ ]:
print(suicide_counts)

Suicide
Not Suicide post           1127
Potential Suicide post      660
Name: count, dtype: int64


In [ ]:
data['Suicide'].value_counts().index.values

array(['Not Suicide post', 'Potential Suicide post '], dtype=object)

In [ ]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [ ]:
train_data['Suicide'].value_counts().index.values

array(['Not Suicide post', 'Potential Suicide post '], dtype=object)

In [ ]:
# DATA VISUALIZATION

In [ ]:
plx.bar(train_data,
        x=train_data['Suicide'].value_counts().index.values,
        y=train_data['Suicide'].value_counts().values, # Use .values to get the counts
        color=['Suicide','Not Suicide'])

In [ ]:
# DATA CLEANING

In [ ]:
data.columns

Index(['Tweet', 'Suicide'], dtype='object')

In [ ]:
def clean_Tweet(Tweets):
    Tweet_length = []
    cleaned_Tweets = []

    for sent in tqdm(Tweets):
        if isinstance(sent, str):
            sent = sent.lower()
            sent = nfx.remove_special_characters(sent)
            cleaned_Tweets.append(sent)
            Tweet_length.append(len(sent))
        else:
            cleaned_Tweets.append("")  # or handle non-string cases as needed
            Tweet_length.append(0)

    return cleaned_Tweets, Tweet_length

In [ ]:
cleaned_train_Tweet, train_Tweet_length = clean_Tweet(train_data['Tweet'])

100%|██████████| 1429/1429 [00:00<00:00, 132167.42it/s]


In [ ]:
cleaned_test_Tweet, test_Tweet_length = clean_Tweet(test_data['Tweet'])

100%|██████████| 358/358 [00:00<00:00, 101484.24it/s]


In [ ]:
train_data['cleaned_Tweet'] = cleaned_train_Tweet
train_data['Tweet_length'] = train_Tweet_length

In [ ]:
test_data['cleaned_Tweet'] = cleaned_test_Tweet
test_data['Tweet_length'] = test_Tweet_length

In [ ]:
print(train_data.head())

                                                  Tweet  \
1518         @sputnick42 welcome &amp; good to see you!   
512   @michaelgrainger @Scyranth *gigglessssssss* Ya...   
1305  @MissSmashlee oh not alot, uni, working, same ...   
1215  I tried to be good and not buy food but i dead...   
7     @jakepaul @jamescharles @LoganPaul Just becaus...   

                      Suicide  \
1518         Not Suicide post   
512          Not Suicide post   
1305         Not Suicide post   
1215  Potential Suicide post    
7     Potential Suicide post    

                                          cleaned_Tweet  Tweet_length  
1518             sputnick42 welcome amp good to see you            38  
512   michaelgrainger scyranth gigglessssssss yall a...            61  
1305  misssmashlee oh not alot uni working same as j...           130  
1215  i tried to be good and not buy food but i dead...            85  
7     jakepaul jamescharles loganpaul just because s...           130  


In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_Tweet)

In [ ]:
cleaned_train_Tweet

['sputnick42 welcome amp good to see you',
 'michaelgrainger scyranth gigglessssssss yall are awesomesauce',
 'misssmashlee oh not alot uni working same as just about everyone handed in the first assignment in over a year on friday was crazy',
 'i tried to be good and not buy food but i dead ass want chipotles chicken and rice rn',
 'jakepaul jamescharles loganpaul just because shane made a series on you doesnt mean people take you seriously y httpstcod5ra0nua0u',
 'markgeollli pls dont bring this image to my mind im not ready to die',
 'i am so miserable ive never been this miserable in my life he is simply baby httpstco6phayb9oha',
 'i am so tired of everything please just let me die',
 '200 words over with still more to write lame burritos for tea',
 'julouille3 let me hate myself god fucking damn it',
 'animeesayi want to say ill never be active again not quite sure i dont have any reason to be on here anymore anyways bye everyone take care',
 'bcuban popehat not a lawyer but i was

In [ ]:
train_text_seq = tokenizer.texts_to_sequences(cleaned_train_Tweet)
train_text_pad = pad_sequences(train_text_seq, maxlen=50)

test_text_seq = tokenizer.texts_to_sequences(cleaned_test_Tweet)
test_text_pad = pad_sequences(test_text_seq, maxlen=50)

In [ ]:
train_text_pad

array([[   0,    0,    0, ...,    2,   91,   10],
       [   0,    0,    0, ...,  340,   57, 1674],
       [   0,    0,    0, ...,  617,   32,  537],
       ...,
       [  22,    9,    3, ...,   52,   27,  164],
       [   0,    0,    0, ...,  341,   38,  340],
       [   0,    0,    0, ...,    1,   40,   28]], dtype=int32)

In [ ]:
# GLOVE EMBEDDINGS

In [ ]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['Suicide'])
test_output=lbl_target.transform(test_data['Suicide'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
with open(r'/content/drive/My Drive/glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)


In [ ]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.18732999,  0.40595001, -0.51174003, ...,  0.16495   ,
         0.18757001,  0.53873998],
       [ 0.31924   ,  0.06316   , -0.27858001, ...,  0.082745  ,
         0.097801  ,  0.25044999],
       ...,
       [-0.59500003, -0.012561  ,  0.14918999, ...,  0.44632   ,
        -0.42645001,  0.50085998],
       [ 0.22649001,  0.057262  , -0.38793999, ...,  0.10541   ,
         0.17787001,  0.13725001],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [ ]:
# Keras Sequential Model Construction

In [ ]:
model=Sequential()
model.add(Input(shape=(50,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 50, 300)           1628100   
                                                                 
 lstm_3 (LSTM)               (None, 50, 20)            25680     
                                                                 
 global_max_pooling1d_3 (Gl  (None, 20)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 256)               5376      
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1659413 (6.33 MB)
Trainable params: 31313 (122.32 KB)
Non-trainable params: 1628100 (6.21 MB)
____________

In [ ]:
# Model Training and Evaluation

In [ ]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

Epoch 1/20
6/6 [==============================] - 4s 241ms/step - loss: 0.6902 - accuracy: 0.5514 - val_loss: 0.6734 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 2/20
6/6 [==============================] - 1s 158ms/step - loss: 0.6710 - accuracy: 0.6277 - val_loss: 0.6588 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 3/20
6/6 [==============================] - 1s 142ms/step - loss: 0.6618 - accuracy: 0.6277 - val_loss: 0.6517 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 4/20
6/6 [==============================] - 1s 135ms/step - loss: 0.6561 - accuracy: 0.6277 - val_loss: 0.6451 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 5/20
6/6 [==============================] - 1s 151ms/step - loss: 0.6505 - accuracy: 0.6277 - val_loss: 0.6389 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 6/20
6/6 [==============================] - 1s 220ms/step - loss: 0.6442 - accuracy: 0.6277 - val_loss: 0.6316 - val_accuracy: 0.6425 - lr: 0.1000
Epoch 7/20
6/6 [==============================] - 1s 121ms/step - loss: 0.6369 - a

In [ ]:
# Use model.predict() and np.argmax() for prediction
print('TESTING DATA CLASSIFICATION REPORT \n \n')
predictions = (model.predict(test_text_pad) > 0.5).astype("int32") # Assuming binary classification
print(classification_report(test_output, predictions,
                            target_names=lbl_target.inverse_transform([0,1])))

print('TRAINING DATA CLASSIFICATION REPORT \n \n')
predictions = (model.predict(train_text_pad) > 0.5).astype("int32") # Assuming binary classification
print(classification_report(train_output, predictions,
                           target_names=lbl_target.inverse_transform([0,1])))

TESTING DATA CLASSIFICATION REPORT 
 

12/12 [==============================] - 1s 14ms/step
                         precision    recall  f1-score   support

       Not Suicide post       0.93      0.81      0.87       230
Potential Suicide post        0.72      0.88      0.80       128

               accuracy                           0.84       358
              macro avg       0.83      0.85      0.83       358
           weighted avg       0.85      0.84      0.84       358

TRAINING DATA CLASSIFICATION REPORT 
 

45/45 [==============================] - 1s 14ms/step
                         precision    recall  f1-score   support

       Not Suicide post       0.92      0.82      0.87       897
Potential Suicide post        0.75      0.88      0.81       532

               accuracy                           0.84      1429
              macro avg       0.83      0.85      0.84      1429
           weighted avg       0.86      0.84      0.85      1429



In [ ]:
twt = ['i am happy']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)

prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
else:
    print("Non Suicide Post")

1/1 [==============================] - 0s 44ms/step
0.16659504
Non Suicide Post


In [ ]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
token_form = pickle.load(open('tokenizer.pkl', 'rb'))

In [ ]:
from keras.models import load_model

In [ ]:
model_form = load_model("model.h5")

In [ ]:
twt = ['Through these past years thoughts of suicide, fear, anxiety I’m so close to my limit']
twt = token_form.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)


prediction = model_form.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
elif (prediction == 1):
    print("Non Suicide Post")

1/1 [==============================] - 1s 741ms/step
0.6677336
Potential Suicide Post
